***Can only retrieve up to 1000 data points from Flipside API***

In [201]:
import pandas as pd
import numpy as np 
import requests
import json
import time
from flipside import Flipside
import os
import traceback
from dotenv import load_dotenv
import datetime as dt
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor



from sql.sql_scripts import mints, sales, eth_price

In [13]:
load_dotenv()

True

In [318]:
pd.options.display.float_format = '{:,.2f}'.format


In [14]:
opensea_api_key = os.getenv('opensea_api_key')
print(opensea_api_key)

c0c9ef9df89042059cee12e5ca9197e9


***Listing Data***

In [15]:
def fetch_listings(api_key, delay_between_requests=1):
    base_url = "https://api.opensea.io/api/v2/listings/collection/3dns-powered-domains/all"
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    params = {"limit": 100} 

    listings = []
    page_count = 0

    while True:
        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            fetched_listings = data.get("listings", [])
            listings.extend(fetched_listings)
            page_count += 1
            
            # Extract and print the cursor
            next_cursor = data.get("next")
            print(f"Page {page_count}, Cursor: {next_cursor}, Listings Fetched: {len(fetched_listings)}")
            
            if next_cursor:
                params['next'] = next_cursor  # Update the 'next' parameter for the next request
            else:
                break  # No more pages to fetch
                
            # Implementing delay
            time.sleep(delay_between_requests)
            
        else:
            print(f"Failed to fetch data: {response.status_code}")
            break

    print(f"Total pages fetched: {page_count}")
    print(f"Total listings fetched: {len(listings)}")
    
    df = pd.DataFrame(listings)
    return df

***Descriptions***

In [16]:
def save_last_identifier(identifier):
    with open("last_identifier.txt", "w") as file:
        file.write(identifier)

def load_last_identifier():
    try:
        with open("last_identifier.txt", "r") as file:
            return file.read().strip()
    except FileNotFoundError:
        return None

In [17]:
def fetch_all_descriptions(api_key, delay_between_requests=1):
    base_url = "https://api.opensea.io/api/v2/collection/3dns-powered-domains/nfts"
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    params = {"limit": 100}

    all_descriptions = []

    page_count = 0
    last_identifier = load_last_identifier()

    while True:
        if last_identifier:
            params['last_identifier'] = last_identifier

        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            fetched_descriptions = data.get("nfts", [])
            
            if not fetched_descriptions:
                break

            # Process only name and identifier for each description
            for description in fetched_descriptions:
                processed_description = {
                    "name": description.get('name'),
                    "identifier": description.get('identifier')
                }
                all_descriptions.append(processed_description)
            
            # Update the last_identifier to the latest one fetched
            last_identifier = fetched_descriptions[-1].get('identifier')
            save_last_identifier(last_identifier)
            
            page_count += 1
            next_cursor = data.get("next")
            print(f"Page {page_count}, Cursor: {next_cursor} Descriptions Fetched: {len(fetched_descriptions)}, total fetched: {len(all_descriptions)}")
            
            if next_cursor:
                params['next'] = next_cursor
            else:
                break  # No more pages to fetch

            time.sleep(delay_between_requests)
        else:
            print(f"Failed to fetch data: {response.status_code}")
            break

    print(f"Total pages fetched: {page_count}, Total descriptions fetched: {len(all_descriptions)}")
    
    # Save the processed descriptions to a file
    df = pd.DataFrame(all_descriptions)
    return df

***Events***

In [18]:
import json
import os

def save_last_timestamp(event_type, timestamp):
    data = {}
    if os.path.exists("last_timestamps.json"):
        with open("last_timestamps.json", "r") as file:
            data = json.load(file)
    data[event_type] = timestamp
    with open("last_timestamps.json", "w") as file:
        json.dump(data, file)

def load_last_timestamp(event_type):
    if os.path.exists("last_timestamps.json"):
        with open("last_timestamps.json", "r") as file:
            data = json.load(file)
        return data.get(event_type, None)
    return None

In [19]:
def fetch_event_type(api_key, event_type, all_events, params, headers):
    base_url = f"https://api.opensea.io/api/v2/events/collection/3dns-powered-domains"
    params['event_type'] = event_type
    
    # Load the last timestamp/identifier
    last_timestamp = load_last_timestamp(event_type)
    if last_timestamp:
        params['occurred_after'] = last_timestamp
    
    page_count = 0
    while True:
        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            fetched_events = data.get("asset_events", [])
            all_events.extend(fetched_events)
            
            if fetched_events:
                # Update the last timestamp/identifier to the latest one fetched
                last_event_time = fetched_events[-1].get("created_date")
                save_last_timestamp(event_type, last_event_time)
            
            page_count += 1
            next_cursor = data.get("next")
            print(f"Fetching {event_type}: Page {page_count}, Events Fetched: {len(fetched_events)}, Total Events: {len(all_events)}, next cursor: {next_cursor}")
            
            if next_cursor:
                params['next'] = next_cursor
            else:
                break  # No more pages to fetch

            time.sleep(1)  # Delay between requests
        else:
            print(f"Failed to fetch {event_type} data: HTTP {response.status_code}, Response: {response.text}")
            break

def fetch_all_events(api_key):
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    params = {
        "limit": 50  # Adjust the limit as needed
    }

    all_events = []

    # Fetch listings
    fetch_event_type(api_key, "listing", all_events, params.copy(), headers)

    # Fetch sales
    fetch_event_type(api_key, "sale", all_events, params.copy(), headers)

    # Save the fetched events to a DataFrame
    print(f"Total events fetched: {len(all_events)}")
    df = pd.DataFrame(all_events)
    return df 

***Flipside Data***

In [20]:
flipside_api_key = os.getenv("FLIPSIDE_API_KEY")
flipside = Flipside(flipside_api_key, "https://api-v2.flipsidecrypto.xyz")

In [21]:
def flipside_api_results(query):
  query_result_set = flipside.query(query)
  # what page are we starting on?
  current_page_number = 1

  # How many records do we want to return in the page?
  page_size = 1000

  # set total pages to 1 higher than the `current_page_number` until
  # we receive the total pages from `get_query_results` given the 
  # provided `page_size` (total_pages is dynamically determined by the API 
  # based on the `page_size` you provide)

  total_pages = 2


  # we'll store all the page results in `all_rows`
  all_rows = []

  while current_page_number <= total_pages:
    results = flipside.get_query_results(
      query_result_set.query_id,
      page_number=current_page_number,
      page_size=page_size
    )

    total_pages = results.page.totalPages
    if results.records:
        all_rows = all_rows + results.records
    
    current_page_number += 1

  return pd.DataFrame(all_rows)

query_result_set = flipside.query(sales)

# what page are we starting on?
current_page_number = 1

# How many records do we want to return in the page?
page_size = 1000

# set total pages to 1 higher than the `current_page_number` until
# we receive the total pages from `get_query_results` given the 
# provided `page_size` (total_pages is dynamically determined by the API 
# based on the `page_size` you provide)
total_pages = 2

# we'll store all the page results in `all_rows`
all_rows = []

while current_page_number <= total_pages:
  results = flipside.get_query_results(
    query_result_set.query_id,
    page_number=current_page_number,
    page_size=page_size
  )

  total_pages = results.page.totalPages
  if results.records:
      all_rows = all_rows + results.records
  
  current_page_number += 1

mints_df = pd.DataFrame(all_rows)



def createQueryRun(sql):
    url = "https://api-v2.flipsidecrypto.xyz/json-rpc"
    payload = json.dumps({
        "jsonrpc": "2.0",
        "method": "createQueryRun",
        "params": [{
            "resultTTLHours": 1,
            "maxAgeMinutes": 0,
            "sql": sql,
            "tags": {"source": "streamlit-demo", "env": "test"},
            "dataSource": "snowflake-default",
            "dataProvider": "flipside"
        }],
        "id": 1
    })
    headers = {'Content-Type': 'application/json', 'x-api-key': flipside_api_key}
    response = requests.post(url, headers=headers, data=payload)
    response_data = response.json()

    # Check for errors in the response
    if 'error' in response_data:
        error_message = response_data['error'].get('message', 'No error message provided')
        print("Full response data for debugging:", response_data)
        raise Exception(f"Error: {error_message}")

    if 'result' not in response_data or 'queryRun' not in response_data['result'] or 'id' not in response_data['result']['queryRun']:
        print("Unexpected response structure:", response_data)
        raise Exception("Unexpected response structure")

    query_run_id = response_data['result']['queryRun']['id']
    return response_data, query_run_id

#@st.cache_data(ttl='15m')
def getQueryResults(query_run_id, attempts=10, delay=30):
    url = "https://api-v2.flipsidecrypto.xyz/json-rpc"
    payload = json.dumps({
        "jsonrpc": "2.0",
        "method": "getQueryRunResults",
        "params": [{"queryRunId": query_run_id, "format": "json", "page": {"number": 1, "size": 10000}}],
        "id": 1
    })
    headers = {'Content-Type': 'application/json', 'x-api-key': flipside_api_key}

    for attempt in range(attempts):
        response = requests.post(url, headers=headers, data=payload)
        resp_json = response.json()
        if 'result' in resp_json:
            return resp_json  # Data is ready
        elif 'error' in resp_json and 'message' in resp_json['error'] and 'not yet completed' in resp_json['error']['message']:
            time.sleep(delay)  # Wait for a bit before retrying
        else:
            print("Unexpected response or error:", resp_json)
            break  # Break on unexpected error
    return None  # Return None if data isn't ready after all attempts


def retrieve_flipside_data(query):
    try:
        response_data, q_id = createQueryRun(query)
        if q_id:
            df_json = getQueryResults(q_id)
            if df_json:
                df = pd.DataFrame(df_json['result']['rows'])
                print(f"data fetched: {df.head()}")
                return df
            else:
                print('Failed to get results')
        else:
            print('Failed to create query run')
    except Exception as e:
        print(f"Error in fetching data: {e}")
        traceback.print_exc()
    

***Data Retrieval/Processing***

In [22]:
mint_df = flipside_api_results(mints)

In [23]:
mint_df

,day,tokenid,tx_hash,__row_index
0,2024-07-05T20:00:00.000Z,8.985197e+76,0x23a609ddf25089081798088ca8c5714be7b76d58ebf5...,0
1,2024-07-05T20:00:00.000Z,8.985197e+76,0x23a609ddf25089081798088ca8c5714be7b76d58ebf5...,1
2,2024-07-05T20:00:00.000Z,6.778101e+76,0xe8c87f0320802853e22ead69cef537c3fa80d5b9370c...,2
3,2024-07-05T20:00:00.000Z,6.778101e+76,0xe8c87f0320802853e22ead69cef537c3fa80d5b9370c...,3
4,2024-07-05T16:00:00.000Z,1.013928e+77,0x388ad6ef8ededcf3b310ddbf8c84ccaa2e835afd4b7a...,4
...,...,...,...,...
32667,2023-10-04T02:00:00.000Z,7.307980e+76,0x981dd9c93e0419a227128b7ee95c620e7dbc88fe92e8...,32667
32668,2023-10-04T02:00:00.000Z,9.704968e+76,0xd7ece008b97cad61f02396def394751c3fa9a4c2844a...,32668
32669,2023-10-04T02:00:00.000Z,9.704968e+76,0xd7ece008b97cad61f02396def394751c3fa9a4c2844a...,32669
32670,2023-10-04T02:00:00.000Z,7.787567e+76,0x64f70f46174e15001ff109ce325f9d0e0d7a571a22ed...,32670


In [24]:
sales_df = flipside_api_results(sales)

In [25]:
eth_usd_df = flipside_api_results(eth_price)

In [422]:
events_df = fetch_all_events(api_key= opensea_api_key)

Fetching listing: Page 1, Events Fetched: 50, Total Events: 50, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA2LTI5KzIxJTNBMzQlM0ExNi43MTU4OTMmLWV2ZW50X3R5cGU9Y3JlYXRlZCYtcGs9MjM4OTI3NTUxNjk=
Fetching listing: Page 2, Events Fetched: 50, Total Events: 100, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA2LTI0KzIzJTNBNTElM0E0OC4yNTk5NzQmLWV2ZW50X3R5cGU9Y3JlYXRlZCYtcGs9MjM3NTc0NjYwNDY=
Fetching listing: Page 3, Events Fetched: 50, Total Events: 150, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA2LTIyKzE0JTNBNTklM0EzNy43NjU0MDkmLWV2ZW50X3R5cGU9Y3JlYXRlZCYtcGs9MjM2ODkwMzU4MTA=
Fetching listing: Page 4, Events Fetched: 50, Total Events: 200, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA2LTIyKzEyJTNBNTklM0EyNS4zODMxNjAmLWV2ZW50X3R5cGU9Y3JlYXRlZCYtcGs9MjM2ODY2ODk4OTA=
Fetching listing: Page 5, Events Fetched: 50, Total Events: 250, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA2LTE4KzE2JTNBNDUlM0EyNS40MzcxMTUmLWV2ZW50X3R5cGU9Y3JlYXRlZCYtcGs9MjM1ODU4MTMwMTk=
Fetching listing: Page 6, Events Fetched: 50, 

In [27]:
descriptions_df = fetch_all_descriptions(api_key= opensea_api_key)

Page 1, Cursor: LXBrPTE4OTc4MTE2ODE= Descriptions Fetched: 100, total fetched: 100
Page 2, Cursor: LXBrPTE4ODczNDUxNjg= Descriptions Fetched: 100, total fetched: 200
Page 3, Cursor: LXBrPTE4NzgxNDU2MzE= Descriptions Fetched: 100, total fetched: 300
Page 4, Cursor: LXBrPTE4NjkzMDk1ODA= Descriptions Fetched: 100, total fetched: 400
Page 5, Cursor: LXBrPTE4NjIzMTEwNDg= Descriptions Fetched: 100, total fetched: 500
Page 6, Cursor: LXBrPTE4NTA5MDM1ODA= Descriptions Fetched: 100, total fetched: 600
Page 7, Cursor: LXBrPTE4NDA5MzA5NTQ= Descriptions Fetched: 100, total fetched: 700
Page 8, Cursor: LXBrPTE4MzcwMTczNzA= Descriptions Fetched: 100, total fetched: 800
Page 9, Cursor: LXBrPTE4MzQzODYyMDI= Descriptions Fetched: 100, total fetched: 900
Page 10, Cursor: LXBrPTE4MzA4MDQwNjA= Descriptions Fetched: 100, total fetched: 1000
Page 11, Cursor: LXBrPTE4Mjc3NTEwMDU= Descriptions Fetched: 100, total fetched: 1100
Page 12, Cursor: LXBrPTE4MjQ4NzY3ODA= Descriptions Fetched: 100, total fetched: 120

In [28]:
listings_df = fetch_listings(api_key= opensea_api_key, delay_between_requests=1)

Page 1, Cursor: cGs9MTc0ODI3MTAyMDYmY3JlYXRlZF9kYXRlPTIwMjQtMDMtMDYrMDMlM0E1MiUzQTMyLjU4ODA4Nw==, Listings Fetched: 100
Page 2, Cursor: cGs9MTg2NDAxMDQxODMmY3JlYXRlZF9kYXRlPTIwMjQtMDQtMjgrMDklM0E1MiUzQTAxLjAwNzE2MQ==, Listings Fetched: 100
Page 3, Cursor: cGs9MTg3OTI1MTI0NjEmY3JlYXRlZF9kYXRlPTIwMjQtMDUtMDQrMTUlM0E1NyUzQTA4LjEzMjQ4Nw==, Listings Fetched: 100
Page 4, Cursor: cGs9MTkzMDU3ODA5MzAmY3JlYXRlZF9kYXRlPTIwMjQtMDUtMjUrMjMlM0E0MCUzQTU2LjI4MzYzNA==, Listings Fetched: 100
Page 5, Cursor: cGs9MTk3MjA5NDcxMDgmY3JlYXRlZF9kYXRlPTIwMjQtMDYtMTMrMDYlM0E1OSUzQTE4LjE4NjQ0MQ==, Listings Fetched: 100
Page 6, Cursor: cGs9MTk5MzMwODgxNTImY3JlYXRlZF9kYXRlPTIwMjQtMDYtMjIrMTUlM0EwNSUzQTA4LjIzMDk0OA==, Listings Fetched: 100
Page 7, Cursor: cGs9MjAyMTc2ODEyNzUmY3JlYXRlZF9kYXRlPTIwMjQtMDctMDQrMTUlM0E0MiUzQTMwLjM1NzA1Nw==, Listings Fetched: 100
Page 8, Cursor: None, Listings Fetched: 4
Total pages fetched: 8
Total listings fetched: 704


In [29]:
def unpack_protocol_data(row):
    protocol_data = row['protocol_data']
    parameters = protocol_data.get('parameters', {})
    consideration = parameters.get('consideration', [{}])
    offer = parameters.get('offer', [{}])
    price = row['price']['current']
    
    chain = row['chain']
    order_hash = row['order_hash']
    currency = price.get('currency')
    price_string = price.get('value')
    price_in_eth = float(price_string) / (10 ** price.get('decimals', 18))
    primary_recipient = consideration[0].get('recipient') if consideration else None
    identifier_or_criteria = offer[0].get('identifierOrCriteria') if offer else None
    start_time = parameters.get('startTime')
    end_time = parameters.get('endTime')
    
    return pd.Series([
        chain, order_hash, currency, price_string, price_in_eth, 
        primary_recipient, identifier_or_criteria, start_time, end_time
    ])

In [30]:
unpacked_columns = listings_df.apply(unpack_protocol_data, axis=1)
unpacked_columns.columns = [
    'chain', 'order_hash', 'currency', 'price_string', 'price_in_eth', 
    'primary_recipient', 'identifier_or_criteria', 'start_time', 'end_time'
]
listings_df = pd.concat([listings_df, unpacked_columns], axis=1)
listings_df = listings_df.drop(columns=['protocol_data'])


In [31]:
listings_df['identifier_or_criteria'] = listings_df['identifier_or_criteria'].astype(float)
listings_df.rename(columns={'identifier_or_criteria':'tokenid'}, inplace=True)

In [262]:
listings_df.drop(columns=['price_string','currency','primary_recipient','chain','price','protocol_address','type'], inplace=True)

,order_hash,order_hash,price_in_eth,tokenid,start_time,end_time
0,0x902ff72f57b579d6a2fac92a0714ca5274172ccdeb51...,0x902ff72f57b579d6a2fac92a0714ca5274172ccdeb51...,5.0000,2.497682e+76,1705460459,1721181652
1,0x472882ed6ef6df3e646a968fe399a794cb2d80783c0d...,0x472882ed6ef6df3e646a968fe399a794cb2d80783c0d...,8.0000,1.033482e+77,1705973310,1721694506
2,0x3040ad3f37193e684cb2baa0811d21cd345108df5a2a...,0x3040ad3f37193e684cb2baa0811d21cd345108df5a2a...,3.0000,1.022958e+77,1706042993,1721767789
3,0xb62518c46df32c07046a53f3df61112cc4460bc648ac...,0xb62518c46df32c07046a53f3df61112cc4460bc648ac...,3.0000,7.527018e+76,1706043120,1721767916
4,0xe4c2e069a35f4b943e449d8e510f952ff3cb2fc133d3...,0xe4c2e069a35f4b943e449d8e510f952ff3cb2fc133d3...,8.0000,1.535191e+76,1706069852,1721791047
...,...,...,...,...,...,...
699,0x76a6ca76dd9bdc10d146fffadcb276220828ab704e06...,0x76a6ca76dd9bdc10d146fffadcb276220828ab704e06...,0.7175,5.585128e+76,1720107680,1735659732
700,0xe88ff918367aad4e2bca9ed8ae1d6e6ca91f11ad6c21...,0xe88ff918367aad4e2bca9ed8ae1d6e6ca91f11ad6c21...,3.0000,4.753433e+76,1720176067,1728124852
701,0x8a76980edeb088fa648230cf0fcbd36ea9beedb2c924...,0x8a76980edeb088fa648230cf0fcbd36ea9beedb2c924...,5.0000,2.836047e+76,1720184000,1720443248
702,0xc18fc47b36024151d34e664da343db1238ce525e6b63...,0xc18fc47b36024151d34e664da343db1238ce525e6b63...,15.0000,1.315335e+76,1720197367,1736094964


In [32]:
descriptions_df.rename(columns={'identifier':'tokenid'}, inplace=True)

In [33]:
descriptions_df['tokenid'] = descriptions_df['tokenid'].astype(float)

In [277]:
listings_with_names = listings_df.merge(descriptions_df, how='left', on='tokenid')
box_listings = listings_with_names[listings_with_names['name'].str.endswith('.box')]
box_listings

,order_hash,order_hash,price_in_eth,tokenid,start_time,end_time,name
1,0x472882ed6ef6df3e646a968fe399a794cb2d80783c0d...,0x472882ed6ef6df3e646a968fe399a794cb2d80783c0d...,8.0000,1.033482e+77,1705973310,1721694506,open.box
2,0x3040ad3f37193e684cb2baa0811d21cd345108df5a2a...,0x3040ad3f37193e684cb2baa0811d21cd345108df5a2a...,3.0000,1.022958e+77,1706042993,1721767789,jewelry.box
3,0xb62518c46df32c07046a53f3df61112cc4460bc648ac...,0xb62518c46df32c07046a53f3df61112cc4460bc648ac...,3.0000,7.527018e+76,1706043120,1721767916,gay.box
4,0xe4c2e069a35f4b943e449d8e510f952ff3cb2fc133d3...,0xe4c2e069a35f4b943e449d8e510f952ff3cb2fc133d3...,8.0000,1.535191e+76,1706069852,1721791047,power.box
5,0x8bc67ab0f6355d8c78a317dee43dd93aad665ffa66d0...,0x8bc67ab0f6355d8c78a317dee43dd93aad665ffa66d0...,69.0000,9.204574e+76,1706083009,1721804202,dan.box
...,...,...,...,...,...,...,...
697,0x3cd4aa6db0fee75df48765a474f0473a9c7b4f9bcc70...,0x3cd4aa6db0fee75df48765a474f0473a9c7b4f9bcc70...,0.6150,4.113206e+76,1720107680,1735659732,83.box
698,0x8d159ceea17ebc82b9396ed64daae18ca1c74b859339...,0x8d159ceea17ebc82b9396ed64daae18ca1c74b859339...,0.8200,4.124373e+76,1720107680,1735659732,84.box
699,0x76a6ca76dd9bdc10d146fffadcb276220828ab704e06...,0x76a6ca76dd9bdc10d146fffadcb276220828ab704e06...,0.7175,5.585128e+76,1720107680,1735659732,29.box
700,0xe88ff918367aad4e2bca9ed8ae1d6e6ca91f11ad6c21...,0xe88ff918367aad4e2bca9ed8ae1d6e6ca91f11ad6c21...,3.0000,4.753433e+76,1720176067,1728124852,zksync.box


In [440]:
events_df_copy = events_df.copy()
events_df_copy

,event_type,order_hash,order_type,chain,protocol_address,start_date,expiration_date,asset,quantity,maker,taker,payment,criteria,event_timestamp,is_private_listing,closing_date,nft,seller,buyer,transaction
0,order,0x983e688bda44a880edfa1cb92a339c986fff7d1d8a89...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,237,423.00","1,722,915,815.00",{'identifier': '123558974577682993811495050087...,1,0x5e7e26fabb9b813e2faa24234e4ff7a8c7bf9cad,,"{'quantity': '3800000000000000', 'token_addres...",{},1720237438,False,NaN,NaN,NaN,NaN,NaN
1,order,0xb52bcae9f69517283444522735fc9cd0dcdd2df7e90d...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,237,231.00","1,722,915,586.00",{'identifier': '360118525401874510073463955038...,1,0x5e7e26fabb9b813e2faa24234e4ff7a8c7bf9cad,,"{'quantity': '4100000000000000', 'token_addres...",{},1720237244,False,NaN,NaN,NaN,NaN,NaN
2,order,0x3bf6f73f8456237ca2aac3cc5f7a2f9f22b8a1b4ce9a...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,234,176.00","1,735,786,228.00",{'identifier': '102926579013154103204029678028...,1,0x22739f9bbc10cf6412a67a3f135cd89e3e9e35f7,,"{'quantity': '10250000000000000000', 'token_ad...",{},1720234240,False,NaN,NaN,NaN,NaN,NaN
3,order,0x2017a8bc278f9ed47ae209d6f643e6555e042752b0fc...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,234,176.00","1,735,786,228.00",{'identifier': '984146680085041125667580852961...,1,0x22739f9bbc10cf6412a67a3f135cd89e3e9e35f7,,"{'quantity': '10250000000000000000', 'token_ad...",{},1720234240,False,NaN,NaN,NaN,NaN,NaN
4,order,0xb67bca48b982ea3330efbcbf0a5fc9de922d42c1b9a7...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,234,002.00","1,720,237,555.00",{'identifier': '412437292300569613373968868435...,1,0x22739f9bbc10cf6412a67a3f135cd89e3e9e35f7,,"{'quantity': '410000000000000000', 'token_addr...",{},1720234067,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2923,sale,0x8bde44f2d46537cf8d025cc498eaf7bdeadce3631f15...,NaN,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,NaN,NaN,NaN,1,NaN,NaN,"{'quantity': '1000000000000000', 'token_addres...",NaN,1706970257,NaN,"1,706,970,257.00",{'identifier': '260923785748676206958654735781...,0xcc0e0b9ebd4ce352f5a50b3ae77ea01202c284ee,0xc48a8ac51df7b71e500a5580b89e47ab26ca1472,0x60e952a8e0fa8d052791b2e4a22fd52eb3c759511e62...
2924,sale,0xc0547b5b8d575e3ba16b529ee2530a35c437bc75fa93...,NaN,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,NaN,NaN,NaN,1,NaN,NaN,"{'quantity': '110000000000000000', 'token_addr...",NaN,1706653465,NaN,"1,706,653,465.00",{'identifier': '566479911085778512201556021651...,0x240ad467a71210629d71d4de22ebde27951c83fc,0x82eb45562f991329ed2867f43fc60f0ba52c3dab,0x6672189deb7829838aabba20ddeb66a3cb5fee5b55a2...
2925,sale,0xff4b9bdd3f140f8211864ef34e0a07ea0a92eaf98f7d...,NaN,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,NaN,NaN,NaN,1,NaN,NaN,"{'quantity': '10000000000000000', 'token_addre...",NaN,1706203283,NaN,"1,706,203,283.00",{'identifier': '364706086468987072255605061503...,0x7b363822d744143d51c69b0757165c987a6e4ef4,0xe68e8cc7ff772b026c062b9cc28246676d044947,0xd8e71f4d52c43fa770b9aa7ad7d5cfb073f04c7495e0...
2926,sale,0x03188e15aaf9596307117e30355f6b2ac79665fbd1c2...,NaN,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,NaN,NaN,NaN,1,NaN,NaN,"{'quantity': '150000000000000000', 'token_addr...",NaN,1704080589,NaN,"1,704,080,589.00",{'identifier': '992340878683635895851755157396...,0x31b32020fb1bdf0228c58b80590f07b235f2b0ce,0x64233eaa064ef0d54ff1a963933d0d2d46ab5829,0x186d1ceacf620bab710d11cf928b79e5dd46790ff1d5...


In [454]:
bids = events_df_copy[events_df_copy['event_type'] == 'order']
sales = events_df_copy[events_df_copy['event_type'] == 'sale']

bids

,event_type,order_hash,order_type,chain,protocol_address,start_date,expiration_date,asset,quantity,maker,taker,payment,criteria,event_timestamp,is_private_listing,closing_date,nft,seller,buyer,transaction
0,order,0x983e688bda44a880edfa1cb92a339c986fff7d1d8a89...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,237,423.00","1,722,915,815.00",{'identifier': '123558974577682993811495050087...,1,0x5e7e26fabb9b813e2faa24234e4ff7a8c7bf9cad,,"{'quantity': '3800000000000000', 'token_addres...",{},1720237438,False,NaN,NaN,NaN,NaN,NaN
1,order,0xb52bcae9f69517283444522735fc9cd0dcdd2df7e90d...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,237,231.00","1,722,915,586.00",{'identifier': '360118525401874510073463955038...,1,0x5e7e26fabb9b813e2faa24234e4ff7a8c7bf9cad,,"{'quantity': '4100000000000000', 'token_addres...",{},1720237244,False,NaN,NaN,NaN,NaN,NaN
2,order,0x3bf6f73f8456237ca2aac3cc5f7a2f9f22b8a1b4ce9a...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,234,176.00","1,735,786,228.00",{'identifier': '102926579013154103204029678028...,1,0x22739f9bbc10cf6412a67a3f135cd89e3e9e35f7,,"{'quantity': '10250000000000000000', 'token_ad...",{},1720234240,False,NaN,NaN,NaN,NaN,NaN
3,order,0x2017a8bc278f9ed47ae209d6f643e6555e042752b0fc...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,234,176.00","1,735,786,228.00",{'identifier': '984146680085041125667580852961...,1,0x22739f9bbc10cf6412a67a3f135cd89e3e9e35f7,,"{'quantity': '10250000000000000000', 'token_ad...",{},1720234240,False,NaN,NaN,NaN,NaN,NaN
4,order,0xb67bca48b982ea3330efbcbf0a5fc9de922d42c1b9a7...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,234,002.00","1,720,237,555.00",{'identifier': '412437292300569613373968868435...,1,0x22739f9bbc10cf6412a67a3f135cd89e3e9e35f7,,"{'quantity': '410000000000000000', 'token_addr...",{},1720234067,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2834,order,0x3bbb21c5a1766b47ea91b2dafd73e7cb28198b41b3c6...,listing,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,"1,703,615,460.00","1,706,293,860.00",{'identifier': '596691372334868032834900148779...,1,0xec00481d13a4028e0b76b031cf5c863604cb2d4b,,"{'quantity': '3000000000000000000', 'token_add...",{},1703615466,False,NaN,NaN,NaN,NaN,NaN
2835,order,0xa2931b385c3bd5c237a44cc39a452e9f12499ba6cafd...,listing,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,"1,703,615,375.00","1,706,293,775.00",{'identifier': '178657174778441322562715723560...,1,0xec00481d13a4028e0b76b031cf5c863604cb2d4b,,"{'quantity': '3000000000000000000', 'token_add...",{},1703615382,False,NaN,NaN,NaN,NaN,NaN
2836,order,0x0fff804500518519b60164f5a10ef76e63da9ee77775...,listing,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,"1,703,615,237.00","1,706,293,637.00",{'identifier': '887409359338823895470224549460...,1,0x64233eaa064ef0d54ff1a963933d0d2d46ab5829,,"{'quantity': '3000000000000000000', 'token_add...",{},1703615249,False,NaN,NaN,NaN,NaN,NaN
2837,order,0x3e6ab1b4b98dd45a1cd97fc141659077293095161497...,listing,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,"1,703,607,296.00","1,704,212,091.00",{'identifier': '117315147600270829665702926801...,1,0x7af4ca674c86ac74d15ae668ae7ffbe4f7ddafbf,,"{'quantity': '1000000000000000000', 'token_add...",{},1703607304,False,NaN,NaN,NaN,NaN,NaN


In [455]:
bids['identifier'] = bids['asset'].apply(lambda x: x.get('identifier') if isinstance(x, dict) else None)
sales['identifier'] = sales['nft'].apply(lambda x: x.get('identifier') if isinstance(x, dict) else None)

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\2668839905.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\2668839905.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [456]:
bids['identifier'] = bids['identifier'].astype(float)
bids.rename(columns={'identifier':'tokenid'}, inplace=True) 


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\46667634.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\46667634.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [457]:
bids = bids.merge(descriptions_df, how='left', on='tokenid')

In [458]:
bids.columns

Index(['event_type', 'order_hash', 'order_type', 'chain', 'protocol_address',
       'start_date', 'expiration_date', 'asset', 'quantity', 'maker', 'taker',
       'payment', 'criteria', 'event_timestamp', 'is_private_listing',
       'closing_date', 'nft', 'seller', 'buyer', 'transaction', 'tokenid',
       'name'],
      dtype='object')

In [459]:
bids.drop(columns=['protocol_address','chain','maker','criteria','is_private_listing','closing_date','nft','seller','buyer'], inplace=True)

In [460]:
sales['identifier'] = sales['identifier'].astype(float)
sales.rename(columns={'identifier':'tokenid'}, inplace=True)


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\3642358893.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\3642358893.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [461]:
sales = sales.merge(descriptions_df, how='left', on='tokenid')
sales.drop(columns=['order_type', 'chain','start_date','expiration_date','asset','maker','is_private_listing','protocol_address','criteria'], inplace=True)

In [462]:
bids

,event_type,order_hash,order_type,start_date,expiration_date,asset,quantity,taker,payment,event_timestamp,transaction,tokenid,name
0,order,0x983e688bda44a880edfa1cb92a339c986fff7d1d8a89...,listing,"1,720,237,423.00","1,722,915,815.00",{'identifier': '123558974577682993811495050087...,1,,"{'quantity': '3800000000000000', 'token_addres...",1720237438,NaN,"12,355,897,457,768,299,701,688,231,177,476,341,...",art-if-icialgaming.xyz
1,order,0xb52bcae9f69517283444522735fc9cd0dcdd2df7e90d...,listing,"1,720,237,231.00","1,722,915,586.00",{'identifier': '360118525401874510073463955038...,1,,"{'quantity': '4100000000000000', 'token_addres...",1720237244,NaN,"36,011,852,540,187,452,254,927,620,002,392,477,...",wwwdigital.quest
2,order,0x3bf6f73f8456237ca2aac3cc5f7a2f9f22b8a1b4ce9a...,listing,"1,720,234,176.00","1,735,786,228.00",{'identifier': '102926579013154103204029678028...,1,,"{'quantity': '10250000000000000000', 'token_ad...",1720234240,NaN,"102,926,579,013,154,100,286,505,544,828,164,204...",dollar.box
3,order,0x2017a8bc278f9ed47ae209d6f643e6555e042752b0fc...,listing,"1,720,234,176.00","1,735,786,228.00",{'identifier': '984146680085041125667580852961...,1,,"{'quantity': '10250000000000000000', 'token_ad...",1720234240,NaN,"98,414,668,008,504,107,161,480,855,001,558,634,...",ok.box
4,order,0xb67bca48b982ea3330efbcbf0a5fc9de922d42c1b9a7...,listing,"1,720,234,002.00","1,720,237,555.00",{'identifier': '412437292300569613373968868435...,1,,"{'quantity': '410000000000000000', 'token_addr...",1720234067,NaN,"41,243,729,230,056,959,187,156,630,795,533,044,...",84.box
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2834,order,0x3bbb21c5a1766b47ea91b2dafd73e7cb28198b41b3c6...,listing,"1,703,615,460.00","1,706,293,860.00",{'identifier': '596691372334868032834900148779...,1,,"{'quantity': '3000000000000000000', 'token_add...",1703615466,NaN,"59,669,137,233,486,797,576,913,893,823,506,075,...",mintdomains.io
2835,order,0xa2931b385c3bd5c237a44cc39a452e9f12499ba6cafd...,listing,"1,703,615,375.00","1,706,293,775.00",{'identifier': '178657174778441322562715723560...,1,,"{'quantity': '3000000000000000000', 'token_add...",1703615382,NaN,"17,865,717,477,844,132,496,699,921,146,330,333,...",dnsloans.com
2836,order,0x0fff804500518519b60164f5a10ef76e63da9ee77775...,listing,"1,703,615,237.00","1,706,293,637.00",{'identifier': '887409359338823895470224549460...,1,,"{'quantity': '3000000000000000000', 'token_add...",1703615249,NaN,"8,874,093,593,388,239,065,001,647,753,652,567,1...",0ooo0ooo0.ooo
2837,order,0x3e6ab1b4b98dd45a1cd97fc141659077293095161497...,listing,"1,703,607,296.00","1,704,212,091.00",{'identifier': '117315147600270829665702926801...,1,,"{'quantity': '1000000000000000000', 'token_add...",1703607304,NaN,"11,731,514,760,027,082,503,362,540,473,080,970,...",tokentroopers.xyz


In [472]:
box_sales_os = sales[sales['name'].str.endswith('.box')]
box_bids_os = bids[bids['name'].str.endswith('.box')] 

In [473]:

box_bids_os.columns = [f'bid_{col}' if col != 'name' else col for col in box_bids_os.columns]

In [475]:
box_sales_os.columns = [f'sale_{col}' if col != 'name' else col for col in box_sales_os.columns]

In [476]:
box_listings_and_sales = pd.merge(box_bids_os, box_sales_os, how='inner', on='name')

In [478]:
box_listings_and_sales['name']

0          dox.box
1            r.box
2            r.box
3            r.box
4            r.box
          ...     
184       dick.box
185       uber.box
186       uber.box
187    opensea.box
188         vm.box
Name: name, Length: 189, dtype: object

In [34]:
mint_df['tokenid'] = mint_df['tokenid'].astype(float)

In [35]:
mints_with_names = pd.merge(mint_df, descriptions_df, how='left', on='tokenid')

In [36]:
mints_with_names

,day,tokenid,tx_hash,__row_index,name
0,2024-07-05T20:00:00.000Z,8.985197e+76,0x23a609ddf25089081798088ca8c5714be7b76d58ebf5...,0,cryptocreator.tech
1,2024-07-05T20:00:00.000Z,8.985197e+76,0x23a609ddf25089081798088ca8c5714be7b76d58ebf5...,1,cryptocreator.tech
2,2024-07-05T20:00:00.000Z,6.778101e+76,0xe8c87f0320802853e22ead69cef537c3fa80d5b9370c...,2,kawaii.box
3,2024-07-05T20:00:00.000Z,6.778101e+76,0xe8c87f0320802853e22ead69cef537c3fa80d5b9370c...,3,kawaii.box
4,2024-07-05T16:00:00.000Z,1.013928e+77,0x388ad6ef8ededcf3b310ddbf8c84ccaa2e835afd4b7a...,4,seismic.box
...,...,...,...,...,...
32667,2023-10-04T02:00:00.000Z,7.307980e+76,0x981dd9c93e0419a227128b7ee95c620e7dbc88fe92e8...,32667,org
32668,2023-10-04T02:00:00.000Z,9.704968e+76,0xd7ece008b97cad61f02396def394751c3fa9a4c2844a...,32668,net
32669,2023-10-04T02:00:00.000Z,9.704968e+76,0xd7ece008b97cad61f02396def394751c3fa9a4c2844a...,32669,net
32670,2023-10-04T02:00:00.000Z,7.787567e+76,0x64f70f46174e15001ff109ce325f9d0e0d7a571a22ed...,32670,com


In [37]:
mints_with_names_null = mints_with_names[mints_with_names.isnull().any(axis=1)]
print(list(mints_with_names_null['tx_hash']))

['0xb47db1490ab537cbbd227388a32567952c2bf27d5e55864a9415110f2af63ca7', '0xb47db1490ab537cbbd227388a32567952c2bf27d5e55864a9415110f2af63ca7', '0x62f6283e6459d7e2a19c818e515927d159daa57d08ff5b55c79141b7f28b94a3', '0x62f6283e6459d7e2a19c818e515927d159daa57d08ff5b55c79141b7f28b94a3', '0x1f172942f03578e445dfe20e5104a799c87a88a108d4cf095d96c8e841dc8a99', '0x1f172942f03578e445dfe20e5104a799c87a88a108d4cf095d96c8e841dc8a99']


In [38]:
mints_with_names.drop_duplicates('tokenid', inplace=True)

In [39]:
mints_with_names.drop(columns=['__row_index','tx_hash','tokenid'], inplace=True)

In [40]:
mints_with_names.set_index('day', inplace=True)

In [41]:
mints_with_names.index = pd.to_datetime(mints_with_names.index)
mints_with_names.dropna(inplace=True)

In [42]:
box_domains_mints = mints_with_names[mints_with_names['name'].str.endswith('.box')]


In [43]:
box_domains_mints

,name
day,
2024-07-05 20:00:00+00:00,kawaii.box
2024-07-05 16:00:00+00:00,seismic.box
2024-07-05 16:00:00+00:00,location.box
2024-07-05 15:00:00+00:00,sprinter.box
2024-07-05 15:00:00+00:00,certificate.box
...,...
2023-10-31 17:00:00+00:00,duncan.box
2023-10-26 14:00:00+00:00,intercap-dev.box
2023-10-24 19:00:00+00:00,s.box


In [44]:
daily_box_mints = box_domains_mints.resample('D').count()

In [106]:
daily_box_mints.rename(columns={'name':'mints'}, inplace=True)
daily_box_mints_fig = px.bar(daily_box_mints, x=daily_box_mints.index, y='mints', title='Daily Mints')
daily_box_mints_fig.show()

In [46]:
total_box_mints = box_domains_mints.count().iloc[0]
total_box_mints

3599

In [47]:
sales_with_names = pd.merge(sales_df, descriptions_df, how='left', on='tokenid')

In [48]:
sales_with_names.drop_duplicates('tokenid', inplace=True)
sales_with_names.drop(columns=['__row_index','tx_hash','tokenid'], inplace=True)
sales_with_names.set_index('day', inplace=True)
sales_with_names.index = pd.to_datetime(sales_with_names.index)

In [56]:
box_domains_sales = sales_with_names[sales_with_names['name'].str.endswith('.box')]

In [52]:
eth_usd_df.set_index('day', inplace=True)
eth_usd_df.index = pd.to_datetime(eth_usd_df.index)
eth_usd_df.drop(columns=['__row_index'], inplace=True)

In [55]:
eth_usd_df.rename(columns={'price':'eth_usd'}, inplace=True)

In [278]:
box_listings['start_time'] = pd.to_datetime(box_listings['start_time'], unit='s').dt.strftime('%Y-%m-%d %H:00:00')
box_listings['end_time'] = pd.to_datetime(box_listings['end_time'], unit='s').dt.strftime('%Y-%m-%d %H:00:00')

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\1480011625.py:1: FutureWarning:

The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\1480011625.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\1480011625.py:2: FutureWarning:

The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without

In [283]:
eth_usd_df_copy = eth_usd_df.reset_index().copy()
eth_usd_df_copy.rename(columns={'day':'start_time'}, inplace=True)

In [291]:
eth_usd_df_copy['start_time'] = pd.to_datetime(eth_usd_df_copy['start_time']).dt.tz_localize(None)

In [289]:
box_listings['start_time'] = pd.to_datetime(box_listings['start_time']) 

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\1939952577.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [293]:

box_listings = box_listings.merge(eth_usd_df_copy, how='left', on='start_time') 

In [295]:
box_listings.drop(columns=['order_hash'], inplace=True)

In [297]:
box_listings['price_in_usd_start_time'] = box_listings['price_in_eth'] * box_listings['eth_usd']

In [299]:
box_listings.set_index('start_time', inplace=True)
box_listings_max_daily = box_listings['price_in_usd_start_time'].resample('D').max()

In [306]:
box_listings_num_daily = box_listings['name'].resample('D').count()
total_box_listings = box_listings_num_daily.sum()
total_box_listings

519

In [307]:
box_listings_num_daily

start_time
2024-01-23     3
2024-01-24    11
2024-01-25    11
2024-01-26     6
2024-01-27     4
              ..
2024-07-01     0
2024-07-02     5
2024-07-03     0
2024-07-04     8
2024-07-05     2
Freq: D, Name: name, Length: 165, dtype: int64

In [304]:
box_listings_min_daily = box_listings['price_in_usd_start_time'].resample('D').min()
box_listings_avg_daily = box_listings['price_in_usd_start_time'].resample('D').mean()

In [313]:
box_listings_max_daily.fillna(0, inplace=True)
box_listings_min_daily.fillna(0, inplace=True)
box_listings_avg_daily.fillna(0, inplace=True)

In [314]:
box_listing_data = pd.merge(box_listings_num_daily.to_frame('listings'), box_listings_max_daily.to_frame('max_price'), left_index=True,
                            right_index=True, how='inner')

In [316]:
box_listing_data = box_listing_data.merge(box_listings_min_daily.to_frame('min_price'), left_index=True,
                                          right_index=True, how='inner')

box_listing_data = box_listing_data.merge(box_listings_avg_daily.to_frame('avg_price'), left_index=True,
                                          right_index=True, how='inner')

In [319]:
box_listing_data

,listings,max_price,min_price,avg_price
start_time,,,,
2024-01-23,3,"18,508.01","6,582.45","10,557.64"
2024-01-24,11,"931,166.04","3,349.20","160,128.32"
2024-01-25,11,"2,206,503.58",660.15,"882,610.90"
2024-01-26,6,"665,376.23","4,478.13","162,767.74"
2024-01-27,4,"1,590,895.35","680,521.15","946,259.95"
...,...,...,...,...
2024-07-01,0,0.00,0.00,0.00
2024-07-02,5,"119,676.90","3,541.23","68,411.18"
2024-07-03,0,0.00,0.00,0.00


In [381]:
monthly_listings = box_listings['name'].resample('M').count()
monthly_listings

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\1847853540.py:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



start_time
2024-01-31     36
2024-02-29     17
2024-03-31     11
2024-04-30     94
2024-05-31    145
2024-06-30    201
2024-07-31     15
Freq: ME, Name: name, dtype: int64

In [382]:
def monthly_listings_growth_rate(listings):
    previous_month = listings.shift(1)
    growth_rate = ((listings - previous_month) / previous_month) * 100
    return growth_rate

In [383]:
listings_growth_rate = monthly_listings_growth_rate(monthly_listings)


In [384]:
listings_growth_rate.dropna(inplace=True)
listings_growth_rate

start_time
2024-02-29   -52.78
2024-03-31   -35.29
2024-04-30   754.55
2024-05-31    54.26
2024-06-30    38.62
2024-07-31   -92.54
Freq: ME, Name: name, dtype: float64

In [57]:
box_domains_sales = box_domains_sales.merge(eth_usd_df, left_index=True, right_index=True, how='left')

In [59]:
box_domains_sales['price_usd'] = box_domains_sales['price'] * box_domains_sales['eth_usd']
box_domains_sales.rename(columns={'price':'price_eth'}, inplace=True)

In [61]:
box_domains_sales.drop(columns=['eth_usd'], inplace=True)


,price_eth,name,price_usd
day,,,
2024-04-17 18:00:00+00:00,0.0100,mewtwo.box,29.610981
2024-04-24 04:00:00+00:00,0.0290,0000.box,93.923366
2024-04-25 21:00:00+00:00,0.1000,express.box,317.327581
2024-04-27 16:00:00+00:00,10.0000,vm.box,31481.468013
2024-04-27 00:00:00+00:00,0.3675,jane.box,1151.714038
2024-04-28 07:00:00+00:00,0.0950,good.box,314.608157
2024-04-28 01:00:00+00:00,0.0690,man.box,224.604619
2024-05-07 22:00:00+00:00,0.0550,onyx.box,167.833449
2024-02-03 22:00:00+00:00,0.0144,696.box,33.088886


In [113]:
box_domains_sales.sort_index(inplace=True)
box_domains_mints.sort_index(inplace=True)

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\3245620862.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [81]:
box_domains_sales = box_domains_sales[['name', 'price_usd','price_eth']]


In [158]:
box_domains_sales

,name,price_usd,price_eth
day,,,
2024-02-03 22:00:00+00:00,696.box,33.088886,0.0144
2024-02-05 04:00:00+00:00,song.box,45.601150,0.0200
2024-02-10 19:00:00+00:00,to.box,96.974190,0.0390
2024-02-20 12:00:00+00:00,uae.box,203.189586,0.0700
2024-02-22 04:00:00+00:00,08.box,116.739602,0.0400
2024-02-24 04:00:00+00:00,404.box,203.681142,0.0700
2024-02-25 02:00:00+00:00,010.box,119.084361,0.0400
2024-02-25 03:00:00+00:00,70.box,269.172017,0.0900
2024-02-25 03:00:00+00:00,40.box,209.356013,0.0700


In [82]:
max_eth_sale = box_domains_sales['price_eth'].max()
max_usd_sale = box_domains_sales['price_usd'].max()

# Retrieve the corresponding timestamps
max_eth_sale_row = box_domains_sales.loc[box_domains_sales['price_eth'].idxmax()]
max_usd_sale_row = box_domains_sales.loc[box_domains_sales['price_usd'].idxmax()]

# Display the results
print(f"Maximum sale: \n {max_eth_sale_row}")


Maximum sale: 
 name               vm.box
price_usd    31481.468013
price_eth            10.0
Name: 2024-04-27 16:00:00+00:00, dtype: object


In [310]:
total_box_sales = box_domains_sales['name'].count()
print(f'total .box domain sales as of {dt.datetime.today()} : {total_box_sales}')

total .box domain sales as of 2024-07-05 22:56:22.861562 : 21


In [96]:
daily_box_sales = box_domains_sales['name'].resample('D').count()
daily_box_sales

day
2024-02-03 00:00:00+00:00    1
2024-02-04 00:00:00+00:00    0
2024-02-05 00:00:00+00:00    1
2024-02-06 00:00:00+00:00    0
2024-02-07 00:00:00+00:00    0
                            ..
2024-05-03 00:00:00+00:00    0
2024-05-04 00:00:00+00:00    0
2024-05-05 00:00:00+00:00    0
2024-05-06 00:00:00+00:00    0
2024-05-07 00:00:00+00:00    1
Freq: D, Name: name, Length: 95, dtype: int64

In [255]:
daily_box_vol = box_domains_sales['price_usd'].resample('D').sum()
cumulative_box_vol = daily_box_vol.cumsum()

In [107]:
daily_box_sales_fig = px.bar(daily_box_sales.to_frame('sales'), x=daily_box_sales.index, y='sales', title='Daily Sales')
daily_box_sales_fig.show()

In [111]:
latest_box_domains_sales = box_domains_sales.iloc[-10:] 
latest_box_domains_sales

,name,price_usd,price_eth
day,,,
2024-02-25 03:00:00+00:00,015.box,104.678007,0.0350
2024-02-25 07:00:00+00:00,38.box,301.731691,0.1000
2024-04-17 18:00:00+00:00,mewtwo.box,29.610981,0.0100
2024-04-24 04:00:00+00:00,0000.box,93.923366,0.0290
2024-04-25 21:00:00+00:00,express.box,317.327581,0.1000
2024-04-27 00:00:00+00:00,jane.box,1151.714038,0.3675
2024-04-27 16:00:00+00:00,vm.box,31481.468013,10.0000
2024-04-28 01:00:00+00:00,man.box,224.604619,0.0690
2024-04-28 07:00:00+00:00,good.box,314.608157,0.0950


In [259]:
latest_box_domains_mints = box_domains_mints.iloc[-10:]
latest_box_domains_mints

,name
day,
2024-07-03 21:00:00+00:00,dogematic.box
2024-07-04 00:00:00+00:00,engineer.box
2024-07-04 01:00:00+00:00,loudpunx.box
2024-07-04 08:00:00+00:00,trader.box
2024-07-04 18:00:00+00:00,fork.box
2024-07-05 15:00:00+00:00,certificate.box
2024-07-05 15:00:00+00:00,sprinter.box
2024-07-05 16:00:00+00:00,location.box
2024-07-05 16:00:00+00:00,seismic.box


In [260]:
cumulative_box_mints = daily_box_mints.cumsum()
cumulative_box_mints

,mints
day,
2023-10-04 00:00:00+00:00,1
2023-10-05 00:00:00+00:00,1
2023-10-06 00:00:00+00:00,1
2023-10-07 00:00:00+00:00,1
2023-10-08 00:00:00+00:00,1
...,...
2024-07-01 00:00:00+00:00,3563
2024-07-02 00:00:00+00:00,3583
2024-07-03 00:00:00+00:00,3590


In [117]:
cumulative_box_sales = daily_box_sales.cumsum()
cumulative_box_sales

day
2024-02-03 00:00:00+00:00     1
2024-02-04 00:00:00+00:00     1
2024-02-05 00:00:00+00:00     2
2024-02-06 00:00:00+00:00     2
2024-02-07 00:00:00+00:00     2
                             ..
2024-05-03 00:00:00+00:00    20
2024-05-04 00:00:00+00:00    20
2024-05-05 00:00:00+00:00    20
2024-05-06 00:00:00+00:00    20
2024-05-07 00:00:00+00:00    21
Freq: D, Name: name, Length: 95, dtype: int64

In [248]:
monthly_max_sold = box_domains_sales['price_usd'].resample('M').max()
monthly_min_sold = box_domains_sales['price_usd'].resample('M').min()
monthly_avg_sold = box_domains_sales['price_usd'].resample('M').mean()
monthly_volume_usd = box_domains_sales['price_usd'].resample('M').sum()

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\547997385.py:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\547997385.py:2: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\547997385.py:3: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\547997385.py:4: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [249]:
monthly_max_sold.fillna(0, inplace=True)
monthly_min_sold.fillna(0, inplace=True)
monthly_avg_sold.fillna(0, inplace=True)
monthly_volume_usd.fillna(0, inplace=True)

In [250]:
monthly_box_sales_metrics = pd.merge(monthly_max_sold.to_frame('max_price'), monthly_min_sold.to_frame('min_price'), left_index=True, right_index=True, how='inner')

In [251]:
monthly_box_sales_metrics.merge(monthly_avg_sold.to_frame('avg_price'), left_index=True, right_index=True, how='inner')

,max_price,min_price,avg_price
day,,,
2024-02-29 00:00:00+00:00,301.731691,33.088886,162.081129
2024-03-31 00:00:00+00:00,0.000000,0.000000,0.000000
2024-04-30 00:00:00+00:00,31481.468013,29.610981,4801.893822
2024-05-31 00:00:00+00:00,167.833449,167.833449,167.833449


In [252]:
monthly_box_sales_metrics.merge(monthly_volume_usd.to_frame('volume_usd'), left_index=True, right_index=True, how='inner')

,max_price,min_price,volume_usd
day,,,
2024-02-29 00:00:00+00:00,301.731691,33.088886,2107.054671
2024-03-31 00:00:00+00:00,0.000000,0.000000,0.000000
2024-04-30 00:00:00+00:00,31481.468013,29.610981,33613.256755
2024-05-31 00:00:00+00:00,167.833449,167.833449,167.833449


In [139]:
daily_sales_metrics = pd.merge(cumulative_box_sales.to_frame('cumulative_sales'), daily_box_sales.to_frame('daily_sales'), 
                               left_index=True, right_index=True, how='left')

In [257]:
daily_sales_metrics = daily_sales_metrics.merge(daily_box_vol.to_frame('vol_usd'), left_index=True, right_index=True, how='inner')
daily_sales_metrics = daily_sales_metrics.merge(cumulative_box_vol.to_frame('cumulative_vol'), left_index=True, right_index=True, how='inner')

In [258]:
daily_sales_metrics

,cumulative_sales,daily_sales,vol_usd,cumulative_vol
day,,,,
2024-02-03 00:00:00+00:00,1,1,33.088886,33.088886
2024-02-04 00:00:00+00:00,1,0,0.000000,33.088886
2024-02-05 00:00:00+00:00,2,1,45.601150,78.690036
2024-02-06 00:00:00+00:00,2,0,0.000000,78.690036
2024-02-07 00:00:00+00:00,2,0,0.000000,78.690036
...,...,...,...,...
2024-05-03 00:00:00+00:00,20,0,0.000000,35720.311426
2024-05-04 00:00:00+00:00,20,0,0.000000,35720.311426
2024-05-05 00:00:00+00:00,20,0,0.000000,35720.311426


In [385]:
monthly_listings = monthly_listings.to_frame('listings')


In [386]:
monthly_listings

,listings
start_time,
2024-01-31,36
2024-02-29,17
2024-03-31,11
2024-04-30,94
2024-05-31,145
2024-06-30,201
2024-07-31,15


In [387]:
monthly_sales = box_domains_sales['name'].resample('M').count()
monthly_sales = monthly_sales.reset_index()
monthly_sales['day'] = pd.to_datetime(monthly_sales['day']).dt.strftime('%Y-%m-%d')
monthly_sales.set_index('day', inplace=True)

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\2388482189.py:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [388]:
monthly_sales

,name
day,
2024-02-29,13
2024-03-31,0
2024-04-30,7
2024-05-31,1


In [391]:
monthly_listings.index

DatetimeIndex(['2024-01-31', '2024-02-29', '2024-03-31', '2024-04-30',
               '2024-05-31', '2024-06-30', '2024-07-31'],
              dtype='datetime64[ns]', name='start_time', freq='ME')

In [394]:
monthly_sales.index = pd.to_datetime(monthly_sales.index)

In [395]:
monthly_sales_reindexed = monthly_sales.reindex(monthly_listings.index).fillna(0)
monthly_sales_reindexed


,name
start_time,
2024-01-31,0.00
2024-02-29,13.00
2024-03-31,0.00
2024-04-30,7.00
2024-05-31,1.00
2024-06-30,0.00
2024-07-31,0.00


In [396]:
monthly_listings['sales'] = monthly_sales_reindexed['name']

In [397]:
monthly_listings

,listings,sales
start_time,,
2024-01-31,36,0.00
2024-02-29,17,13.00
2024-03-31,11,0.00
2024-04-30,94,7.00
2024-05-31,145,1.00
2024-06-30,201,0.00
2024-07-31,15,0.00


In [399]:
monthly_listings['listings_to_sales_ratio'] = monthly_listings['listings'] / monthly_listings['sales']
monthly_listings['listings_to_sales_ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)

monthly_listings

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\3641184892.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,listings,sales,listings_to_sales_ratio
start_time,,,
2024-01-31,36,0.00,0.00
2024-02-29,17,13.00,1.31
2024-03-31,11,0.00,0.00
2024-04-30,94,7.00,13.43
2024-05-31,145,1.00,145.00
2024-06-30,201,0.00,0.00
2024-07-31,15,0.00,0.00


In [402]:
monthly_mints = box_domains_mints.resample('M').count()
monthly_mints.reset_index(inplace=True)
monthly_mints['day'] = pd.to_datetime(monthly_mints['day']).dt.strftime('%Y-%m-%d') 
print(monthly_mints)

          day  name
0  2023-10-31     5
1  2023-11-30     2
2  2023-12-31    23
3  2024-01-31  1745
4  2024-02-29   349
5  2024-03-31   162
6  2024-04-30   636
7  2024-05-31   414
8  2024-06-30   221
9  2024-07-31    42


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\2186263740.py:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [407]:
monthly_mints.set_index('day', inplace=True)


KeyError: "None of ['day'] are in the columns"

In [409]:
monthly_mints.index = pd.to_datetime(monthly_mints.index) 
monthly_mints.index

DatetimeIndex(['2023-10-31', '2023-11-30', '2023-12-31', '2024-01-31',
               '2024-02-29', '2024-03-31', '2024-04-30', '2024-05-31',
               '2024-06-30', '2024-07-31'],
              dtype='datetime64[ns]', name='day', freq=None)

In [405]:
monthly_sales_reindexed.index

DatetimeIndex(['2024-01-31', '2024-02-29', '2024-03-31', '2024-04-30',
               '2024-05-31', '2024-06-30', '2024-07-31'],
              dtype='datetime64[ns]', name='start_time', freq='ME')

In [414]:
monthly_mints['sales'] = monthly_sales_reindexed['name'] 
monthly_mints.fillna(0, inplace=True)
monthly_mints['mint_to_sales_ratio'] = monthly_mints['name'] / monthly_mints['sales'] 
monthly_mints['mint_to_sales_ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)
monthly_mints.rename(columns={'name':'mints'}, inplace=True)


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_12140\760772224.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [415]:
monthly_mints

,mints,sales,mint_to_sales_ratio
day,,,
2023-10-31,5,0.00,0.00
2023-11-30,2,0.00,0.00
2023-12-31,23,0.00,0.00
2024-01-31,1745,0.00,0.00
2024-02-29,349,13.00,26.85
2024-03-31,162,0.00,0.00
2024-04-30,636,7.00,90.86
2024-05-31,414,1.00,414.00
2024-06-30,221,0.00,0.00


In [416]:
cumulative_listings_to_sales = total_box_listings / total_box_sales
print(cumulative_listings_to_sales)

24.714285714285715


In [417]:
cumulative_mint_to_sales = total_box_mints / total_box_sales
print(cumulative_mint_to_sales)

171.38095238095238


***Box Domains Valuation Model***

**Data Processing**

In [247]:
domain_path = 'E:/Projects/box_app/data/domain-name-sales.tsv'  
domain_data = pd.read_csv(domain_path, delimiter='\t')


In [156]:
domain_data.set_index('date', inplace=True)
domain_data = domain_data.drop(columns=['venue'])
domain_data.sort_index(inplace=True)

In [161]:
domain_data.index = pd.to_datetime(domain_data.index)
domain_data

,domain,price
date,,
1999-04-01,altavista.com,3250000
1999-04-01,bingo.com,1100000
1999-11-01,fly.com,1500000
1999-12-01,autos.com,2200000
1999-12-01,england.com,2000000
...,...,...
2021-01-01,yes.movie,253
2021-01-01,yopal.com,405
2021-01-01,yougraph.com,1161


In [180]:
domain_data['domain_length'] = domain_data['domain'].apply(len)
domain_data['num_vowels'] = domain_data['domain'].apply(lambda x: sum([1 for char in x if char in 'aeiou']))
domain_data['num_consonants'] = domain_data['domain'].apply(lambda x: sum([1 for char in x if char.isalpha() and char not in 'aeiou']))
domain_data['tld'] = domain_data['domain'].apply(lambda x: x.split('.')[-1])  # Extract TLD


In [181]:
domain_data

,domain,price,domain_length,num_vowels,num_consonants,tld
date,,,,,,
1999-04-01,altavista.com,3250000,13,5,7,com
1999-04-01,bingo.com,1100000,9,3,5,com
1999-11-01,fly.com,1500000,7,1,5,com
1999-12-01,autos.com,2200000,9,4,4,com
1999-12-01,england.com,2000000,11,3,7,com
...,...,...,...,...,...,...
2021-01-01,yes.movie,253,9,4,4,movie
2021-01-01,yopal.com,405,9,3,5,com
2021-01-01,yougraph.com,1161,12,4,7,com


In [160]:
box_domains_sales.columns

Index(['name', 'price_usd', 'price_eth'], dtype='object')

In [162]:
filtered_box = box_domains_sales.drop(columns=['price_eth'])
filtered_box.rename(columns={'name':'domain', 'price_usd':'price'}, inplace=True)


In [182]:
filtered_box['domain_length'] = filtered_box['domain'].apply(len)
filtered_box['num_vowels'] = filtered_box['domain'].apply(lambda x: sum([1 for char in x if char in 'aeiou']))
filtered_box['num_consonants'] = filtered_box['domain'].apply(lambda x: sum([1 for char in x if char.isalpha() and char not in 'aeiou']))
filtered_box['tld'] = filtered_box['domain'].apply(lambda x: x.split('.')[-1])  # Extract TLD


In [165]:
filtered_box.index = filtered_box.index.strftime('%Y-%m-%d')

AttributeError: 'Index' object has no attribute 'strftime'

In [189]:
filtered_box

,domain,price,domain_length,num_vowels,num_consonants,tld
day,,,,,,
2024-02-03,696.box,33.088886,7,1,2,box
2024-02-05,song.box,45.601150,8,2,5,box
2024-02-10,to.box,96.974190,6,2,3,box
2024-02-20,uae.box,203.189586,7,4,2,box
2024-02-22,08.box,116.739602,6,1,2,box
2024-02-24,404.box,203.681142,7,1,2,box
2024-02-25,010.box,119.084361,7,1,2,box
2024-02-25,70.box,269.172017,6,1,2,box
2024-02-25,40.box,209.356013,6,1,2,box


In [186]:
features = ['domain_length', 'num_vowels', 'num_consonants', 'tld']
X = domain_data[features]
y = domain_data['price']

In [196]:
# Preprocess categorical data (TLD) and handle missing values
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), ['domain_length', 'num_vowels', 'num_consonants']),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), ['tld'])
    ]
)

# Create a pipeline with Ridge regression
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'regressor__alpha': [0.1, 1.0, 10.0, 100.0, 1000.0]
}

In [197]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


**Ridge Regression**

In [198]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

# Best model from grid search
best_model = grid_search.best_estimator_

# Predict and evaluate
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Best Alpha: {grid_search.best_params_["regressor__alpha"]}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

Best Alpha: 1000.0
MAE: 2705.0255351946857
MSE: 2852191402.3463864
R²: 0.0003925999206353392


**Random Forest Regressor**

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_split=5, random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

**XGBoost**

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

**LightGBM**

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LGBMRegressor(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

**Cat Boost**

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', CatBoostRegressor(iterations=200, depth=5, learning_rate=0.1, random_state=42, verbose=0))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

**Best Model**

In [205]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge(alpha=1000.0))  # Set the best alpha value from grid search
])

In [206]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

MAE: 2705.0255351946857
MSE: 2852191402.3463864
R²: 0.0003925999206353392


In [209]:
filtered_box

,domain,price,domain_length,num_vowels,num_consonants,tld,predicted_price
day,,,,,,,
2024-02-03,696.box,33.088886,7,1,2,box,1168.932145
2024-02-05,song.box,45.601150,8,2,5,box,2173.632242
2024-02-10,to.box,96.974190,6,2,3,box,3066.390943
2024-02-20,uae.box,203.189586,7,4,2,box,3180.120862
2024-02-22,08.box,116.739602,6,1,2,box,2005.653100
2024-02-24,404.box,203.681142,7,1,2,box,1168.932145
2024-02-25,010.box,119.084361,7,1,2,box,1168.932145
2024-02-25,70.box,269.172017,6,1,2,box,2005.653100
2024-02-25,40.box,209.356013,6,1,2,box,2005.653100


In [208]:
box_X = filtered_box[features]

# Predict prices for .box domains using the best model
filtered_box['predicted_price'] = pipeline.predict(box_X)

print(filtered_box[['domain', 'predicted_price']])

                 domain  predicted_price
day                                     
2024-02-03      696.box      1168.932145
2024-02-05     song.box      2173.632242
2024-02-10       to.box      3066.390943
2024-02-20      uae.box      3180.120862
2024-02-22       08.box      2005.653100
2024-02-24      404.box      1168.932145
2024-02-25      010.box      1168.932145
2024-02-25       70.box      2005.653100
2024-02-25       40.box      2005.653100
2024-02-25       ap.box      3066.390943
2024-02-25       75.box      2005.653100
2024-02-25      015.box      1168.932145
2024-02-25       38.box      2005.653100
2024-04-17   mewtwo.box      1560.928176
2024-04-24     0000.box       332.211191
2024-04-25  express.box      1114.548825
2024-04-27     jane.box      2453.686877
2024-04-27       vm.box      2786.336308
2024-04-28      man.box      2620.011592
2024-04-28     good.box      2453.686877
2024-05-07     onyx.box      2173.632242


In [210]:
r2 = r2_score(filtered_box['price'], filtered_box['predicted_price'])
print(f'r2 {r2}')

r2 0.0411345241547072


**.Box Domain Valuator**

In [211]:
filtered_box_2 = filtered_box.drop(columns=['predicted_price'])
filtered_box_2

,domain,price,domain_length,num_vowels,num_consonants,tld
day,,,,,,
2024-02-03,696.box,33.088886,7,1,2,box
2024-02-05,song.box,45.601150,8,2,5,box
2024-02-10,to.box,96.974190,6,2,3,box
2024-02-20,uae.box,203.189586,7,4,2,box
2024-02-22,08.box,116.739602,6,1,2,box
2024-02-24,404.box,203.681142,7,1,2,box
2024-02-25,010.box,119.084361,7,1,2,box
2024-02-25,70.box,269.172017,6,1,2,box
2024-02-25,40.box,209.356013,6,1,2,box


In [212]:
combined_data = pd.concat([domain_data, filtered_box_2], ignore_index=True)

In [213]:
combined_data

,domain,price,domain_length,num_vowels,num_consonants,tld
0,altavista.com,3.250000e+06,13,5,7,com
1,bingo.com,1.100000e+06,9,3,5,com
2,fly.com,1.500000e+06,7,1,5,com
3,autos.com,2.200000e+06,9,4,4,com
4,england.com,2.000000e+06,11,3,7,com
...,...,...,...,...,...,...
348252,jane.box,1.151714e+03,8,3,4,box
348253,vm.box,3.148147e+04,6,1,4,box
348254,man.box,2.246046e+02,7,2,4,box
348255,good.box,3.146082e+02,8,3,4,box


In [214]:
X = combined_data[features]
y = combined_data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')


MAE: 2708.9834718161696
MSE: 2845754262.6555114
R²: 0.0004163141819931715


In [234]:
def model_prep(data):
    data['domain_length'] = data['domain'].apply(len)
    data['num_vowels'] = data['domain'].apply(lambda x: sum([1 for char in x if char in 'aeiou']))
    data['num_consonants'] = data['domain'].apply(lambda x: sum([1 for char in x if char.isalpha() and char not in 'aeiou']))
    data['tld'] = data['domain'].apply(lambda x: x.split('.')[-1]) 
    return data

In [235]:
def value_domain(domain):
    domain_x = domain[features]
    value = pipeline.predict(domain_x)
    print(f'predicted value: {value[0]}')
    return value[0] 

In [244]:
test_domain = 'eth.box' ## for model, just have the person input before .box, have it automatically add .box
test_domain_df = pd.DataFrame({'domain': [test_domain]})
test_domain_processed = model_prep(test_domain_df)
test_domain_value = value_domain(test_domain_processed)

predicted value: 2560.052986420118


In [239]:
test_domain_value

2560.052986420118